Thymio mini project

In [1]:
!pip install --upgrade tdmclient

$ x_{k+1}^T = A \cdot x_k^T + B \cdot y_k^T = \begin{bmatrix} 1 & 0 & 0 & \frac{\text{dt}*TRANSLATION\_CORR*\cos(\theta)}{2} & \frac{\text{dt}*TRANSLATION\_CORR*\cos(\theta)}{2} \\ 0 & 1 & 0 & \frac{\text{dt}*TRANSLATION\_CORR*\sin(\theta)}{2} & \frac{\text{dt}*TRANSLATION\_CORR*\sin(\theta)}{2} \\ 0 & 0 & 1 & \frac{\text{dt}*ROTATION\_CORR\_RIGHT}{4*\text{WHEELS\_DIST}} & -\frac{\text{dt}*ROTATION\_CORR\_LEFT}{4*\text{WHEELS\_DIST}} \\ 0 & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 & 0 \end{bmatrix} \cdot \begin{bmatrix} x \\ y \\ \theta \\ 0 \\ 0 \end{bmatrix} + \begin{bmatrix} 0 & 0 \\ 0 & 0 \\ 0 & 0 \\ 1 & 0 \\ 0 & 1 \end{bmatrix} \cdot \begin{bmatrix} 0 \\ 0 \\ 0 \\ vr \\ vl \end{bmatrix}$

In [15]:
# import libraries
import time
from tdmclient import ClientAsync
import numpy as np
import math

# import project files
import constants as cst
import extended_Kalman_filter as eKf
import vision
import geometry as geo


In [16]:
#from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node 101c2fd7-1c04-4e4c-911d-5b3f3b0c1cc6

In [17]:
await node.set_variables({
            'leds.bottom.left': [0, 0, 0],
            'leds.bottom.right': [0, 0, 0],
            'leds.circle': [0, 0, 0, 0, 0, 0, 0, 0],
            'leds.top': [0, 0, 0],
        })

In [18]:
# Set the motor speeds
async def stop_robot():
    """
    Stop the robot. Set the motor speed of each wheel to zero.
    """
    await node.set_variables({
            "motor.right.target": [0],
            "motor.left.target": [0],
        })

async def move_robot(right_speed, left_speed):
    """
    Turn the robot wheels to the given speeds.
    Parameter: - right_speed   : right wheel speed in [thymio speed]
               - left_speed    : left wheel speed in [thymio speed]
    """
    await node.set_variables({
            "motor.right.target": [right_speed],
            "motor.left.target": [left_speed],
        })

In [19]:
# Get the proximity sensor values
async def get_prox_sensors():
    """
    Get the sensor measurements of a the proximity sensors.
    Returns: - prox_sensors    : numpy array of fron proximity sensors measurements
             - detected        : boolean, true if any sensors measured a higher value than threshold
    """
    
    await node.wait_for_variables({str('prox.horizontal')})
    prox_sensors = np.asarray(list(node['prox.horizontal']))
    prox_sensors = prox_sensors[0:5]

    prox_sensors_bool = (prox_sensors >= np.ones(5) * 20) #cst.THRESHOLD_PROX_SENSOR
    detected = False
    if(any(prox_sensors_bool)):
        detected = True

    return prox_sensors, detected

# Get the motor speed measurement values
async def get_motor_speed_meas():
    """
    Get the motor speed measurements
    Returns: - speed_left_meas    : left wheel speed in [thymio speed]
             - speed_right_meas   : right wheel speed in [thymio speed]
    """
    
    await node.wait_for_variables({str('motor.right.speed')})
    speed_right_meas = node['motor.right.speed']
    await node.wait_for_variables({str('motor.left.speed')})
    speed_left_meas = node['motor.left.speed']

    return speed_right_meas, speed_left_meas

In [20]:
# test thymio
# move the robot forward and get prox values.
await move_robot(50, 50)
dt = 0
old_time = time.time()
while(dt < 2):
    y,d = await get_prox_sensors()
    print(y)
    print(d)
    await client.sleep(0.1)
    dt = time.time() - old_time
await stop_robot()

[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[0 0 0 0 0]
False
[   0 2126 3867 4481 2641]
True
[   0 2126 3867 4481 2641]
True
[   0 4262 4862 5078 3258]
True
[0 0 0 0 0]
False
[0 0 0 0 0]
False


In [ ]:
#test vision and global nav
import vision
import global_navigation as gn

obstacles, goal_centroid = vision.vision_obstacles_and_goal()
robot_centroid, robot_direction = vision.vision_robot()

obstacles = obstacles.tolist()

merge_need = False
visualize = True
shortest_path, obstacles = gn.global_navigation(tuple(robot_centroid), obstacles, tuple(goal_centroid), merge_need, visualize)
print("glob nav path :", shortest_path)

obstacles.append([(4.5,3.5),(4.5,4.5),(6.5,4.5),(6.5,3.5)])
merge_need = True
visualize = True
shortest_path, obstacles = gn.global_navigation(tuple(robot_centroid), obstacles, tuple(goal_centroid), merge_need, visualize)
print("glob nav path :", shortest_path)

obstacles = np.array(obstacles)

In [21]:
# ROBOT CONTROL AND VISION

# state variables
global_state = 'rotation'

### initialisation
# vision (détection robot, obstacles, goal)
#obstacles, goal_centroid, robot_centroid, robot_direction = vision.vision_obstacles_positions()
# global nav

#robot, robot_direction = vision.vision_robot()
#robot_orientation = np.arctan2(robot_direction[0], robot_direction[1])
#robot = robot * 0.308

intermediate_goals = [(0, 10), (10, 10), (10, 0), (0,0)]
robot = [0.0,0.0]
robot_orientation = 0
robot_speed = [0.0,0.0]
speed_correction = 0

#Kfilter = eKf.Kalman(robot[0], robot[1], robot_orientation)


old_time = 0.0

iter = 0
while(True):
    print(iter)
    if intermediate_goals == []:
        await stop_robot()
        print("stop robot")
        break

    # vision et gobale toutes les n iterations (i modulo n == 0)
    
    # get robot sensor values
    #prox_sensors, detected = await get_prox_sensors()
    # test if local (with proximity)
    #if(detected):
    #    global_state = 'local_avoidance'
    
    if iter == 0:                   # first time
        old_time = time.time()
    dt = time.time() - old_time     # get time difference for Kalman
    # print("dt: ", dt)
    # Kalman (with wheel speed and camera position)
    old_time = time.time()

    robot[0] = robot[0] + dt * ((robot_speed[0] + robot_speed[1])/(2.0*18.1818)) * math.cos(robot_orientation)
    robot[1] = robot[1] + dt * ((robot_speed[0] + robot_speed[1])/(2.0*18.1818)) * math.sin(robot_orientation)
    robot_orientation = robot_orientation + dt * (robot_speed[0] - robot_speed[1])/(4.0*cst.WHEELS_DIST)
    print(robot[0]," ", robot[1]," ", robot_orientation)
    
    #Kfilter.dt_update(dt)
    #speed_left_meas, speed_right_meas = await get_motor_speed_meas()
    #Kfilter.measurement_wheels(speed_right_meas*speed_correction / 18.1818, speed_left_meas*speed_correction / 18.1818) 

    #if(iter % 10 == 0):
    #    robot_centroid, robot_direction = vision.vision_robot()
    #    robot_centroid = robot_centroid * 0.308
    #    robot_angle = np.arctan2(robot_direction[0], robot_direction[1])
    #    Kfilter.measurement_position(robot_centroid[0], robot_centroid[1], robot_angle)
    #Kfilter.Kalman_filter()
    #print("Kalman: ", Kfilter.Mu)

    if global_state == 'rotation':
        #angle = Kfilter.Mu[2] - np.arctan2(intermediate_goals[0][1] - Kfilter.Mu[1], intermediate_goals[0][0] - Kfilter.Mu[0])
        angle = np.arctan2(intermediate_goals[0][1] - robot[1], intermediate_goals[0][0] - robot[0]) - robot_orientation
        angle = geo.center_angle(angle)
        #print("rotation", np.arctan2(intermediate_goals[0][1] - Kfilter.Mu[1], intermediate_goals[0][0] - Kfilter.Mu[0]))
        print("rotation :", robot_orientation, np.arctan2(intermediate_goals[0][1] - robot[1], intermediate_goals[0][0] - robot[0]))
        
        if abs(angle) < 0.1:
            global_state = 'translation'
            continue

        # call rotation fct
        if(angle > 0):
            robot_speed = [30, -30]         # counterclockwise
            speed_correction = 12.5
        else:
            robot_speed = [-30, 30]         # clockwise
            speed_correction = 14
        await move_robot(robot_speed[0], robot_speed[1])
        

    elif global_state == 'translation':
        #print("translation ", np.sqrt((intermediate_goals[0][0] - Kfilter.Mu[0])**2 + (intermediate_goals[0][1] - Kfilter.Mu[1])**2))
        print("translation: ", robot[0], " ", robot[1], "  ", math.dist(intermediate_goals[0], robot))
        # if goal reached
            # exit
        #if abs(intermediate_goals[0][0] - Kfilter.Mu[0]) < 1 and abs(intermediate_goals[0][1] - Kfilter.Mu[1]) < 1:
        #if np.sqrt((intermediate_goals[0][0] - Kfilter.Mu[0])**2 + (intermediate_goals[0][1] - Kfilter.Mu[1])**2) < 2:
        if np.sqrt((intermediate_goals[0][0] - robot[0])**2 + (intermediate_goals[0][1] - robot[1])**2) < 2:
            intermediate_goals.pop(0)
            global_state = 'rotation'
            continue
        
        # call translation fct
        robot_speed = [50, 50]
        speed_correction = 0.75
        await move_robot(robot_speed[0], robot_speed[1])

    elif global_state == 'local_avoidance':
        # call local_avoidance fct (= add obstacle on global map and recompute global nav)
        
        global_state = 'rotation'
    time.sleep(0.05)

    iter = iter + 1

0
0.0   0.0   0.0
rotation : 0.0 1.5707963267948966
1
0.0   0.0   0.029760791409400203
rotation : 0.029760791409400203 1.5707963267948966
2
0.0   0.0   0.05459147114907542
rotation : 0.05459147114907542 1.5707963267948966
3
0.0   0.0   0.07982188655484107
rotation : 0.07982188655484107 1.5707963267948966
4
0.0   0.0   0.10462487897565288
rotation : 0.10462487897565288 1.5707963267948966
5
0.0   0.0   0.12946647982443532
rotation : 0.12946647982443532 1.5707963267948966
6
0.0   0.0   0.15410546333559097
rotation : 0.15410546333559097 1.5707963267948966
7
0.0   0.0   0.17872310453845608
rotation : 0.17872310453845608 1.5707963267948966
8
0.0   0.0   0.203203308966852
rotation : 0.203203308966852 1.5707963267948966
9
0.0   0.0   0.22790208939583068
rotation : 0.22790208939583068 1.5707963267948966
10
0.0   0.0   0.25267531794886433
rotation : 0.25267531794886433 1.5707963267948966
11
0.0   0.0   0.27744508558704006
rotation : 0.27744508558704006 1.5707963267948966
12
0.0   0.0   0.3021475

In [13]:
await stop_robot()

In [1]:
#ROBOT CONTROL AND LOCAL

import local_navigation as ln

#test control robot + kalman + local nav

# state variables
global_state = 'rotation'

### initialisation
# vision (détection robot, obstacles, goal)
#obstacles, goal_centroid, robot_centroid, robot_direction = vision.vision_obstacles_positions()
# global nav

intermediate_goals = [(10, 0)]
robot = [0.0,0.0]
robot_orientation = 0
robot_speed = [0.0,0.0]
speed_correction = 1


Kfilter = eKf.Kalman(robot[0], robot[1], robot_orientation)


old_time = 0.0

iter = 0
while(True):
    print(iter)
    if intermediate_goals == []:
        await stop_robot()
        print("stop robot")
        break

    # vision et gobale toutes les n iterations (i modulo n == 0)
    
    # get robot sensor values
    prox_sensors, detected = await get_prox_sensors()
    # test if local (with proximity)
    if(detected):
        global_state = 'local_avoidance'
    
    if iter == 0:                   # first time
        old_time = time.time()
    dt = time.time() - old_time     # get time difference for Kalman
    # print("dt: ", dt)
    # Kalman (with wheel speed and camera position)
    old_time = time.time()

    Kfilter.dt_update(dt)
    speed_left_meas, speed_right_meas = await get_motor_speed_meas()
    speed_right_meas = speed_right_meas * speed_correction
    speed_left_meas = speed_left_meas * speed_correction
    Kfilter.measurement_wheels(speed_right_meas / 18.1818, speed_left_meas / 18.1818)
    Kfilter.Kalman_filter()
    print("Kalman: ", np.degrees(Kfilter.Mu[2]))
    

    if global_state == 'rotation':
        angle = Kfilter.Mu[2] - np.arctan2(intermediate_goals[0][1] - Kfilter.Mu[1], intermediate_goals[0][0] - Kfilter.Mu[0])
        #angle = robot_orientation - np.arctan2(intermediate_goals[0][1] - robot[1], intermediate_goals[0][0] - robot[0])
        print("rotation")
        #print("rotation :", robot_orientation, np.arctan2(intermediate_goals[0][1] - robot[1], intermediate_goals[0][0] - robot[0]))
        
        if abs(angle) < 0.1:
            global_state = 'translation'
            continue

        # call rotation fct
        speed_correction = 20.8
        if(angle > 0):
            robot_speed = [20, -20]         # counterclockwise
        else:
            robot_speed = [-20, 20]         # clockwise
        await move_robot(robot_speed[0], robot_speed[1])
        
 
    elif global_state == 'translation':
        print("translation ", np.sqrt((intermediate_goals[0][0] - Kfilter.Mu[0])**2 + (intermediate_goals[0][1] - Kfilter.Mu[1])**2))
        #print("translation: ", robot[0], " ", robot[1], "  ", math.dist(intermediate_goals[0], robot))
        # if goal reached
            # exit
        #if abs(intermediate_goals[0][0] - Kfilter.Mu[0]) < 1 and abs(intermediate_goals[0][1] - Kfilter.Mu[1]) < 1:
        if np.sqrt((intermediate_goals[0][0] - Kfilter.Mu[0])**2 + (intermediate_goals[0][1] - Kfilter.Mu[1])**2) < 2:
        #if abs((intermediate_goals[0][0] - robot[0])**2 - (intermediate_goals[0][1] - robot[1])**2) < 2:
            intermediate_goals.pop(0)
            global_state = 'rotation'
            continue
        
        # call translation fct
        speed_correction = 0.65
        robot_speed = [50, 50]
        await move_robot(robot_speed[0], robot_speed[1])

    elif global_state == 'local_avoidance':
        # call local_avoidance fct (= add obstacle on global map and recompute global nav)
        robot_direction = (1,0)
        # Object = ln.local_nav([Kfilter.Mu[0], Kfilter.Mu[1]], robot_direction, prox_sensors)
        object_robot_coords = ln.object_robot_coordinates(prox_sensors)
        print(object_robot_coords)
        print([Kfilter.Mu[0], Kfilter.Mu[1]])
        object_ground_coords = ln.robot_to_ground_coords([Kfilter.Mu[0], Kfilter.Mu[1]], robot_direction, object_robot_coords)
        print(object_ground_coords)
        Object = ln.safety_zone(object_ground_coords)
        print(iter)
        print(Object)
        await stop_robot()
        break
        #global_state = 'rotation'

    time.sleep(0.05)

    iter = iter + 1

SyntaxError: invalid syntax (2484705784.py, line 94)

In [9]:
await stop_robot()

In [10]:
#CODE TO CALIBRATE ROTATION/TRANSLATION

#speed_correction = 12.5 #calibration cst for rotation counterclockwise
speed_correction = 14 #calibration cst for rotation counterclockwise
#speed_correction = 0.75 #0.65 #calibration cst for translation


#intermediate_goals = [(0, 1)]
robot = [0.0,0.0]
robot_orientation = 0
robot_speed = [0.0,0.0]

Kfilter = eKf.Kalman(robot[0], robot[1], robot_orientation)

#robot_speed = [30, -30] #robot speed for rotation (counterclockwise)
robot_speed = [-30, 30] #robot speed for rotation (clockwise)
#robot_speed = [50, 50] #robot speed for translation
await move_robot(robot_speed[0], robot_speed[1])

old_time = 0.0

iter = 0
while(True):
    iter = iter + 1
    print(iter)

    speed_right_meas, speed_left_meas = await get_motor_speed_meas()
    speed_right_meas = speed_right_meas * speed_correction
    speed_left_meas = speed_left_meas * speed_correction
    Kfilter.measurement_wheels(speed_right_meas / 18.1818, speed_left_meas / 18.1818)

    if iter == 0:                   # first time
        old_time = time.time()
    dt = time.time() - old_time     # get time difference for Kalman
    old_time = time.time()

    Kfilter.dt_update(dt)
    Kfilter.Kalman_filter()
    print(dt, time.time(), old_time)
    print("Kalman: ", Kfilter.Mu[0][0], Kfilter.Mu[1][0], Kfilter.Mu[2][0], Kfilter.Mu[3][0], Kfilter.Mu[4][0])

    if(abs(Kfilter.Mu[2][0]) > np.pi / 2.0):        #rotation condition
    #if(np.sqrt(Kfilter.Mu[0][0]**2 + Kfilter.Mu[1][0]**2) > 15.0):     #translation condition
        await stop_robot()
        print("stop")
        break

    await client.sleep(0.1)

1
1701705398.843477 1701705398.8450158 1701705398.843477
Kalman:  5.746748370521353e-12 0.0 -5.767343725505867e-08 -10.780010780010791 11.165011165011174
2
0.10219788551330566 1701705398.945675 1701705398.945675
Kalman:  0.023607750304827382 -1.1346173393139733e-09 -0.007234702363970648 -17.24802139516817 17.86402215928132
3
0.1131434440612793 1701705399.0588183 1701705399.0588183
Kalman:  0.061070871544917714 -0.0002590620847425917 -0.018856252049622962 -20.375408682645634 21.086178744690493
4
0.10409402847290039 1701705399.1656563 1701705399.1629124
Kalman:  0.09916776220856102 -0.0009661519719196702 -0.03085799693373342 -21.582670858052555 22.330024617051397
5
0.11986470222473145 1701705399.282777 1701705399.282777
Kalman:  0.17721094292115855 -0.003189330176914899 -0.04500481728568317 -21.092830602585302 22.805866692665955
6
0.10378527641296387 1701705399.3865623 1701705399.3865623
Kalman:  0.278175353972552 -0.007640411843183365 -0.05725128400805053 -20.905686216353693 22.98766304

In [1]:
await stop_robot()

NameError: name 'stop_robot' is not defined